# Decentralized Machine Learning

In [17]:
import warnings
warnings.simplefilter('ignore')

### Client Setup

In [18]:
from web3 import Web3, HTTPProvider, IPCProvider

web3 = Web3(HTTPProvider('http://localhost:8545'))
# web3v2 = Web3(HTTPProvider('https://ropsten.infura.io/c51VIUAGmJ7HIW2QwM3J'))
# web3 = Web3(web3.currentProvider)

PASSPHRASE = 'panda'
TEST_ACCOUNT = web3.eth.coinbase

# web3.eth.blockNumber
acct = web3.personal.newAccount(PASSPHRASE)
print(acct)
print(web3.eth.getBalance(acct))
web3.eth.sendTransaction({"from": TEST_ACCOUNT, "to": acct, "value": 9999999999})
print(web3.eth.getBalance(acct))

0x6dc16b8c98ca4098df49801eba00396213a849f2
0
9999999999


### Contract Deployment

In [20]:
from solc import compile_source


CONTRACT_NAME = 'Greeter.sol'

def compile_source_file(file_path):
   with open(file_path, 'r') as f:
      source = f.read()

   return compile_source(source)


def deploy_contract(w3, contract_interface):
    tx_hash = w3.eth.contract(
        abi=contract_interface['abi'],
        bytecode=contract_interface['bin']).deploy({"from": acct})

    address = w3.eth.getTransactionReceipt(tx_hash)['contractAddress']
    return address


def wait_for_receipt(w3, tx_hash, poll_interval):
   while True:
       tx_receipt = w3.eth.getTransactionReceipt(tx_hash)
       if tx_receipt:
         return tx_receipt
       time.sleep(poll_interval)
    
contract_source_path = CONTRACT_NAME
compiled_sol = compile_source_file(contract_source_path)

contract_id, contract_interface = compiled_sol.popitem()

web3.personal.unlockAccount(acct, PASSPHRASE)

address = deploy_contract(web3, contract_interface)
print("Deployed {0} to: {1}\n".format(contract_id, address))

SolcError: An error occurred during execution
> command: `solc --combined-json abi,asm,ast,bin,bin-runtime,clone-bin,devdoc,interface,opcodes,userdoc`
> return code: `1`
> stderr:

> stdout:
[31mERROR[0m: Unknown option --combined-json

### Event Listening

In [ ]:
contract_obj = web3.eth.contract(
   address=address,
   abi=contract_interface['abi'])

event_filter = contract_obj.pastEvents("Greeting")
event_filter.get()

def txn_digest(txn):
    return "Contract address: {0}\nEvent: {1}\nArg: {2}".format(
        txn['address'], txn['event'], txn['args'])

def event_callback(arg):
    print(txn_digest(arg))

event_filter.watch(event_callback)

# contract_obj.events
# f = web3.utils.filters.LogFilter()
# web3.eth.filter
# f.get_all_entries()



# deploy_txn = token_contract.constructor(web3.eth.coinbase, 12345).transact()
# txn_receipt = web3.eth.getTransactionReceipt(deploy_txn)
# txn_receipt['contractAddress']